### Import important libraries

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import numpy as np
from selenium.webdriver.support.ui import Select
import pandas as pd
import json
from selenium.common.exceptions import TimeoutException

### Fetching news articles using Selenium

In [7]:
allurls = [
    "https://www.moneycontrol.com/news/business/economy/",
    "https://www.moneycontrol.com/news/tags/companies.html",
    "https://www.moneycontrol.com/news/business/mutual-funds/",
    "https://www.moneycontrol.com/news/business/ipo/",
    "https://www.moneycontrol.com/news/business/startups/",
    "https://www.moneycontrol.com/news/business/stocks/",
    "https://www.moneycontrol.com/news/tags/currency.html",
    "https://www.moneycontrol.com/news/business/commodity/",
    "https://www.moneycontrol.com/news/world/"
]



path = '/Users/dungeon_master/Downloads/chromedriver-mac-arm64/chromedriver' # write the path here
# define 'driver' variable /Users/dungeon_master/Downloads/chromedriver-mac-arm64
options = webdriver.ChromeOptions()
options.add_extension(r"C:\Users\parth\OneDrive\Desktop\Blackrock\adblock.crx")
driver = webdriver.Chrome(options=options) 


# open Google Chrome with time.sleep(5)
urlmain = "https://www.moneycontrol.com/news/business/economy/"
driver.get(urlmain)

#obtain window handle of browser in focus
p = driver.current_window_handle
#obtain parent window handle
parent = driver.window_handles[0]
#obtain browser tab window
chld = driver.window_handles[1]
#switch to browser tab
driver.switch_to.window(parent)

data_3d_list = [[] for _ in range(10)]

def extract_article_data(url, i):
    driver.get(url)
    k=0
    for j in range(1,20):
        if(k>=10):
            break
        time.sleep(5)
        try:
            # Click on the image link for the article
            xpath_expression = f'//*[@id="newslist-{j}"]/a/img'
            img_element = driver.find_element(By.XPATH, xpath_expression)
            img_element.click()

            # Find the script element containing the JSON data
            script_element = driver.find_element(By.XPATH, "//script[@type='application/ld+json'][3]")

            # Extract the text content of the script tag
            script_text = script_element.get_attribute("textContent")

            # Load JSON text as a list
            json_list = json.loads(script_text, strict=False)

            # Access the first (and only) dictionary in the list
            json_dict = json_list[0]

            # Check if "isAccessibleForFree" field exists and has a specific value
            if "isAccessibleForFree" in json_dict and json_dict["isAccessibleForFree"]:
                print("Skipping this article as it is inaccessible for free")
                driver.back()
                continue

            # Extract headline and articleBody content
            website="moneycontrol"
            headline = json_dict["headline"]
            article_body = json_dict["articleBody"]
            print(i)
            k=k+1
            article = [website,headline, article_body]
            # Append the headline and articleBody as a pair
            data_3d_list[i].append(article)
        except TimeoutException:
            print("Timeout")
            driver.back()
        
        driver.back()
        # Navigate back to the list of articles
        

# Iterate through the list of URLs and extract data
for i, url in enumerate(allurls):
    extract_article_data(url, i)

print(data_3d_list)
# Close the WebDriver
driver.quit()

0
0
0
0
0
0
Skipping this article as it is inaccessible for free
0
0
0
0
1
1
Skipping this article as it is inaccessible for free
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
Skipping this article as it is inaccessible for free
2
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
Skipping this article as it is inaccessible for free
4
4
4
4
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
7
7
7
Skipping this article as it is inaccessible for free
7
7
7
Skipping this article as it is inaccessible for free
7
7
7
7
8
8
8
8
8
8
8
8
8
8
[[['moneycontrol', 'Average long-term US mortgage rate climbs to 6.96% this week, matching highest level this year', "The average long-term U.S. mortgage rate rose this week to just under 7%, the latest setback for would-be homebuyers already facing affordability challenges due to a housing market limited by a shortage of homes for sale.\n\nMortgage buyer Freddie Mac said Thursday that the average rate on the benchmark 30-year home loan rose to 6.96% from 6.90% last week. A year ago, the rate aver

### Copied the data extracted from Selenium

In [8]:
articles_3d_list_eng = data_3d_list

### Removing the HTML tags in the data

In [9]:
def remove_html_entities(text):
    entities = {
        '\\n': '',
        '\n': ' ',
        '&amp;': '&',
        '&nbsp;': ' ',
        '&quot;': '"',
        '&rdquo;': '"',
        '&mdash;': '-',
        '&ldquo;': '"',
        '&rsquo;': "'",
        # Add more HTML entities here
        '&lt;': '<',    # Less than sign
        '&gt;': '>',    # Greater than sign
        '&copy;': '©',  # Copyright symbol
        # ... and so on
    }
    
    for entity, replacement in entities.items():
        text = text.replace(entity, replacement)
    
    return text

# Loop through the 3D list and remove HTML entities from articles
for i in range(len(articles_3d_list_eng)):
    for j in range(len(articles_3d_list_eng[i])):
        if articles_3d_list_eng[i][j][1] is not None:
            articles_3d_list_eng[i][j][1] = remove_html_entities(articles_3d_list_eng[i][j][1])
        if articles_3d_list_eng[i][j][2] is not None:
            articles_3d_list_eng[i][j][2] = remove_html_entities(articles_3d_list_eng[i][j][2])

### Convert 3D list to 2D list

In [10]:
articles_2d_list_eng = []  # Initialize an empty list

for i in range(len(articles_3d_list_eng)):
    merged_row = []  # Initialize a row for each i
    for j in range(len(articles_3d_list_eng[i])):
        heading = articles_3d_list_eng[i][j][1]
        article = articles_3d_list_eng[i][j][2]
        merged_text = f"{heading}\n{article}"  # Remove one line break here
        merged_row.append(merged_text)
    articles_2d_list_eng.append(merged_row)

# Print the merged articles
for row in articles_2d_list_eng:
    for article in row:
        print(article + "\n")  # Add one line break after each article

Average long-term US mortgage rate climbs to 6.96% this week, matching highest level this year
The average long-term U.S. mortgage rate rose this week to just under 7%, the latest setback for would-be homebuyers already facing affordability challenges due to a housing market limited by a shortage of homes for sale.  Mortgage buyer Freddie Mac said Thursday that the average rate on the benchmark 30-year home loan rose to 6.96% from 6.90% last week. A year ago, the rate averaged 5.22%.  It's the third consecutive weekly increase for the average rate, which now matches its high for the year set on July 13. High rates can add hundreds of dollars a month in costs for borrowers, limiting how much they can afford in a market already unaffordable to many Americans.  "There is no doubt continued high rates will prolong affordability challenges longer than expected, particularly with home prices on the rise again," said Sam Khater, Freddie Mac's chief economist. "However, upward pressure on rate

### function of TTS

In [12]:
from gtts import gTTS
import os
from pydub import AudioSegment

language = 'en'

def speak(text, filename):
    tts = gTTS(text=text, lang=language)
    tts.save(filename)

folders = ["economy", "companies", "mutual-funds","ipo","startups","stocks","currency","commodity","world"]

### Convert TTS_English


In [13]:
# Specify the custom base location for the folders
base_location = r"C:\Users\parth\OneDrive\Desktop\Blackrock_1\demo_final\audio_eng"

for i, (article_group, folder) in enumerate(zip(articles_2d_list_eng, folders)):
    for j in range(len(articles_2d_list_eng[i])):
        folder_name = folder + "_eng" 
        filename = os.path.join(base_location, folder_name, f"{j}.mp3")
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        try:
            speak(articles_2d_list_eng[i][j], filename)
            print(f"Audio file {j} in {folders[j]} saved successfully.")
        except Exception as e:
            print(f"Error processing article {j}: {str(e)}")

Audio file 0 in economy saved successfully.
Audio file 1 in companies saved successfully.
Audio file 2 in mutual-funds saved successfully.
Audio file 3 in ipo saved successfully.
Audio file 4 in startups saved successfully.
Audio file 5 in stocks saved successfully.
Audio file 6 in currency saved successfully.
Audio file 7 in commodity saved successfully.
Audio file 8 in world saved successfully.
Error processing article 9: list index out of range
Audio file 0 in economy saved successfully.
Audio file 1 in companies saved successfully.
Audio file 2 in mutual-funds saved successfully.
Audio file 3 in ipo saved successfully.
Audio file 4 in startups saved successfully.
Audio file 5 in stocks saved successfully.
Audio file 6 in currency saved successfully.
Audio file 7 in commodity saved successfully.
Audio file 8 in world saved successfully.
Error processing article 9: list index out of range
Audio file 0 in economy saved successfully.
Audio file 1 in companies saved successfully.
Audio 

#### make a new 3d list_hindi

In [14]:
articles_3d_list_hin = articles_3d_list_eng

### To convert the english data into hindi

In [15]:
from englisttohindi.englisttohindi import EngtoHindi

def trans_engtohin(data):
    sentences = data.split('.')
    translated_chunks = []

    for sentence in sentences:
        if sentence.strip():
            translation = EngtoHindi(sentence).convert
            translated_chunks.append(translation)

    translated_text = '. '.join(translated_chunks)
    return translated_text

# Assuming articles_3d_list is defined earlier
# articles_3d_list_hin[0][0][1] = trans_engtohin(articles_3d_list_eng[0][0][1])
 
# articles_3d_list_hin[0][0][2] = trans_engtohin(articles_3d_list_eng[0][0][2])

for i in range(len(articles_3d_list_eng)):
    for j in range(len(articles_3d_list_eng[i])):
        articles_3d_list_hin[i][j][1] = trans_engtohin(articles_3d_list_eng[i][j][1])
        articles_3d_list_hin[i][j][2] = trans_engtohin(articles_3d_list_eng[i][j][2])
        print(articles_3d_list_hin[i][j][1])
        print(articles_3d_list_hin[i][j][2])
# print(translated_headline)  # Print the translated headline


औसत दीर्घकालिक अमेरिकी बंधक दर बढ़कर 6 हो गई. इस सप्ताह 96%, जो इस वर्ष के उच्चतम स्तर से मेल खाता है
औसत दीर्घकालिक यू. एस. इस सप्ताह बंधक दर बढ़कर 7% से कम हो गई है, जो भावी घर खरीदने वालों के लिए नवीनतम झटका है, जो बिक्री के लिए घरों की कमी से सीमित आवास बाजार के कारण पहले से ही किफायती चुनौतियों का सामना कर रहे हैं।. बंधक खरीदार फ्रेडी मैक ने गुरुवार को कहा कि बेंचमार्क 30-वर्षीय गृह ऋण पर औसत दर बढ़कर 6 हो गई है. 6 से 96%. पिछले सप्ताह 90%. एक साल पहले यह दर औसतन 5 थी. 22%. यह औसत दर में लगातार तीसरी साप्ताहिक वृद्धि है, जो अब 13 जुलाई को निर्धारित वर्ष के उच्चतम स्तर से मेल खाती है. उच्च दरें उधारकर्ताओं के लिए प्रति माह सैकड़ों डॉलर की लागत जोड़ सकती हैं, जिससे यह सीमित हो जाएगा कि वे उस बाजार में कितना खर्च कर सकते हैं जो पहले से ही कई अमेरिकियों के लिए अप्राप्य है।. फ्रेडी मैक के मुख्य अर्थशास्त्री सैम खातेर ने कहा, "इसमें कोई संदेह नहीं है कि उच्च दरें जारी रहने से सामर्थ्य की चुनौतियां अपेक्षा से अधिक लंबी हो जाएंगी, खासकर घर की कीमतें फिर से बढ़ रही हैं।". "हालांकि, दरों पर

TypeError: sequence item 0: expected str instance, NoneType found

### To convert 3D list into 2D list


In [16]:
articles_2d_list_hin = []  # Initialize an empty list

for i in range(len(articles_3d_list_hin)):
    merged_row = []  # Initialize a row for each i
    for j in range(len(articles_3d_list_hin[i])):
        heading = articles_3d_list_hin[i][j][1]
        article = articles_3d_list_hin[i][j][2]
        merged_text = f"{heading}\n{article}"
        merged_row.append(merged_text)
    articles_2d_list_hin.append(merged_row)

# Print the merged articles
for row in articles_2d_list_hin:
    for article in row:
        print(article + "\n")

औसत दीर्घकालिक अमेरिकी बंधक दर बढ़कर 6 हो गई. इस सप्ताह 96%, जो इस वर्ष के उच्चतम स्तर से मेल खाता है
औसत दीर्घकालिक यू. एस. इस सप्ताह बंधक दर बढ़कर 7% से कम हो गई है, जो भावी घर खरीदने वालों के लिए नवीनतम झटका है, जो बिक्री के लिए घरों की कमी से सीमित आवास बाजार के कारण पहले से ही किफायती चुनौतियों का सामना कर रहे हैं।. बंधक खरीदार फ्रेडी मैक ने गुरुवार को कहा कि बेंचमार्क 30-वर्षीय गृह ऋण पर औसत दर बढ़कर 6 हो गई है. 6 से 96%. पिछले सप्ताह 90%. एक साल पहले यह दर औसतन 5 थी. 22%. यह औसत दर में लगातार तीसरी साप्ताहिक वृद्धि है, जो अब 13 जुलाई को निर्धारित वर्ष के उच्चतम स्तर से मेल खाती है. उच्च दरें उधारकर्ताओं के लिए प्रति माह सैकड़ों डॉलर की लागत जोड़ सकती हैं, जिससे यह सीमित हो जाएगा कि वे उस बाजार में कितना खर्च कर सकते हैं जो पहले से ही कई अमेरिकियों के लिए अप्राप्य है।. फ्रेडी मैक के मुख्य अर्थशास्त्री सैम खातेर ने कहा, "इसमें कोई संदेह नहीं है कि उच्च दरें जारी रहने से सामर्थ्य की चुनौतियां अपेक्षा से अधिक लंबी हो जाएंगी, खासकर घर की कीमतें फिर से बढ़ रही हैं।". "हालांकि, दरों पर

### Convert TTS_Hindi

In [17]:
# Specify the custom base location for the folders
base_location = r"C:\Users\parth\OneDrive\Desktop\Blackrock_1\demo_final\audio_hin"

for i, (article_group, folder) in enumerate(zip(articles_2d_list_hin, folders)):
    for j in range(len(articles_2d_list_hin[i])):
        folder_name = folder + "_hin" 
        filename = os.path.join(base_location, folder_name, f"{j}.mp3")
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        try:
            speak(articles_2d_list_hin[i][j], filename)
            print(f"Audio file {j} in {folders[j]} saved successfully.")
        except Exception as e:
            print(f"Error processing article {j}: {str(e)}")


Audio file 0 in economy saved successfully.
Audio file 1 in companies saved successfully.
Audio file 2 in mutual-funds saved successfully.
Error processing article 3: 429 (Too Many Requests) from TTS API. Probable cause: Unknown
Error processing article 4: 429 (Too Many Requests) from TTS API. Probable cause: Unknown
Error processing article 5: 429 (Too Many Requests) from TTS API. Probable cause: Unknown
Audio file 6 in currency saved successfully.
Error processing article 7: Failed to connect. Probable cause: Unknown
Error processing article 8: Failed to connect. Probable cause: Unknown
Error processing article 9: Failed to connect. Probable cause: Unknown
Error processing article 0: Failed to connect. Probable cause: Unknown
Error processing article 1: Failed to connect. Probable cause: Unknown
Error processing article 2: Failed to connect. Probable cause: Unknown
Error processing article 3: Failed to connect. Probable cause: Unknown
Error processing article 4: Failed to connect. Pr

### To save the transalted data into a txt file


In [ ]:
# file_path = r'C:\Users\parth\OneDrive\Desktop\Blackrock\demo_final\data\output.txt'

# # Open the file in write mode with 'utf-8' encoding
# with open(file_path, 'w', encoding='utf-8') as file:
#     for item in eng_to_hin_data:
#         file.write(item + '\n')

# # Print a confirmation message
# print(f"List elements written to '{file_path}' successfully.")